# Optimizacion de hiperparametros

Setup

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [2]:
# !gsutil cp /home/clas_giulia_s/buckets/b1/datasets/competencia_02_fe_v01_undersampled.parquet /home/clas_giulia_s/datasets/

In [2]:
# base_path = '/home/clas_giulia_s/buckets/b1/'
base_path = '/Users/ignacio/MAESTRIA/DMEF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe_modelito_undersampled.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [261823, 289871, 379817, 481301, 959723]

# data = pd.read_parquet(dataset_path + dataset_file)
data = pd.read_csv(dataset_path + dataset_file)

In [3]:
# meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
#                202001, 202002, 202003, 202004, 202005, 202006,
#                202007, 202008, 202009, 202010, 202011, 202012,
#                202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

meses_train = [202101, 202102, 202103] # dejo afuera 202104 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(51606, 679)

In [4]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [5]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria']
w_train = data['clase_peso']

In [6]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [7]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 200),
    # learning_rate = trial.suggest_float('learning_rate', 0.01, 0.5), # mas bajo, más iteraciones necesita
    learning_rate = trial.suggest_float('learning_rate', 0.1, 0.5), # mas alto para que dure menos para el exp
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1,
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    # print(f"Learning Rate: {params['learning_rate']}, Type: {type(params['learning_rate'])}")
    
    # Use callbacks for early stopping
    rounds = int(50 + 5/learning_rate[0])
    early_stopping_cb = lgb.early_stopping(stopping_rounds= rounds)
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=10000, # modificar, subit y subir... 
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [8]:
storage_name = "sqlite:///" + db_path + "optimizacion_lgbm_modelito_exp.db"
study_name = "competencia1_modelito_exp_lgbm" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-12-01 19:14:41,080] A new study created in RDB with name: competencia1_modelito_exp_lgbm


In [9]:
len(study.trials)

0

In [10]:
study.optimize(objective, n_trials=60)

Training until validation scores don't improve for 72 rounds


[I 2024-12-01 19:15:05,737] Trial 0 finished with value: 597723000.0 and parameters: {'num_leaves': 153, 'learning_rate': 0.2216061378331717, 'min_data_in_leaf': 163, 'feature_fraction': 0.79399140320068, 'bagging_fraction': 0.6946325144921471}. Best is trial 0 with value: 597723000.0.


Early stopping, best iteration is:
[9]	cv_agg's valid gan_eval: 1.19545e+08 + 1.88993e+06
Training until validation scores don't improve for 70 rounds


[I 2024-12-01 19:15:14,472] Trial 1 finished with value: 605262000.0 and parameters: {'num_leaves': 51, 'learning_rate': 0.2431975154587192, 'min_data_in_leaf': 615, 'feature_fraction': 0.42488833179117413, 'bagging_fraction': 0.5031303013504012}. Best is trial 1 with value: 605262000.0.


Early stopping, best iteration is:
[18]	cv_agg's valid gan_eval: 1.21052e+08 + 1.9385e+06
Training until validation scores don't improve for 61 rounds


...

Early stopping, best iteration is:
[10]	cv_agg's valid gan_eval: 1.19328e+08 + 2.09926e+06
Training until validation scores don't improve for 81 rounds


[I 2024-12-01 19:23:54,791] Trial 58 finished with value: 607131000.0 and parameters: {'num_leaves': 32, 'learning_rate': 0.15953244520065413, 'min_data_in_leaf': 1065, 'feature_fraction': 0.5029033330046067, 'bagging_fraction': 0.6648739224718085}. Best is trial 21 with value: 609161000.0.


Early stopping, best iteration is:
[26]	cv_agg's valid gan_eval: 1.21426e+08 + 2.01983e+06
Training until validation scores don't improve for 93 rounds


[I 2024-12-01 19:24:07,854] Trial 59 finished with value: 605234000.0 and parameters: {'num_leaves': 28, 'learning_rate': 0.11488904542177002, 'min_data_in_leaf': 1342, 'feature_fraction': 0.5036445122671557, 'bagging_fraction': 0.665152566217836}. Best is trial 21 with value: 609161000.0.


Early stopping, best iteration is:
[55]	cv_agg's valid gan_eval: 1.21047e+08 + 1.86952e+06


In [11]:
optuna.visualization.plot_optimization_history(study)

In [12]:
plot_param_importances(study)

In [13]:
plot_slice(study)

In [14]:
plot_contour(study)

In [15]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [16]:
study.best_trial.params

{'num_leaves': 129,
 'learning_rate': 0.14285485549019958,
 'min_data_in_leaf': 1190,
 'feature_fraction': 0.41197212976185593,
 'bagging_fraction': 0.6194842328414374}

In [18]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

50